# Задача
Известно что компания конкурент выпустила 1000 прототипов игр, из которых было 5 успешных. Наша компания выпустила 200 прототипов из которых ни один не был успешен.

Какова вероятность что следующий (201й) наш прототип будет успешен? Какими методами можно решать данную задачу? Предложите как минимум 2 варианта. Если у вас есть какой-то вариант решения задачи для которого требуются доп. данные - перечислите что требуется и метод решения.

# Решение

Для корректного решения задачи не хватает данных, так что стоит сразу уточнить, что придется прибегать к допущениям, которые могут не работать в реальной жизни.

Пространство вероятных исходов состоит из двух элементов. Допускаем, что не будет полууспехов/полунеудач или прототипов с неизвестным итогом. Однако еще стоит помнить, что это неравновероятные исходы, как при подбрасывании «честной» монеты.

In [3]:
# пространство исходов
results_space = {'success', 'fail'}

Приводится статистика по успешности прототипов у конкурента (1):

In [1]:
# всего прототипов
count_prototypes_1 = 1000
# из них успешных:
count_success_1 = 5

In [2]:
print(f'Вероятность успеха у конкурентов: {count_success_1 / count_prototypes_1}')

Вероятность успеха у конкурентов: 0.005


Вот только результаты конкурента нам никак не помогут, ведь в задаче не сказано, что вероятность успеха выпуска прототипов у двух компаний одинакова. Это даже не стыкуется со здравым смыслом, так как вероятности скорее всего не могут быть одинаковыми для разных компаний, разных прототипов, разных разработчиков. Мы даже не знаем, сколько всего выпустим прототипов. Следовательно, придется вводить новые допущения. Можем, например, взять два варианта:
1. Вероятность успеха конкурента равна нашей (=0.005)
2. Мы настолько в себе уверены, что наша вероятность выпустить успешный прототип в два раза выше конкурентов (=0.01)

Так как каждый прототип представляет из себя независимое от других событие без ограничений на возможное количество, то вероятность не должна меняться хоть для первого, хоть для двести первого прототипа. Но давайте смоделируем биномиальное распределение, чтобы убедиться.

In [68]:
import numpy as np

In [45]:
def eval_success_prob(iterations: int, win_rate: float) -> str:
    np.random.seed(42)
    success_count = np.random.binomial(iterations, win_rate)
    prob_success = success_count / iterations
    return f'Вероятность успешных прототипов на {iterations} итерациях с вероятностью {win_rate} равна {prob_success}'

In [23]:
# посмотрим для первого варианта на разном количестве итераций
for iters in [500, 1000, 5000]:
    print(eval_success_prob(iters, 0.005))

Вероятность успешных прототипов на 500 итерациях с вероятностью 0.005 равна 0.004
Вероятность успешных прототипов на 1000 итерациях с вероятностью 0.005 равна 0.004
Вероятность успешных прототипов на 5000 итерациях с вероятностью 0.005 равна 0.0046


In [24]:
for iters in [500, 1000, 5000]:
    print(eval_success_prob(iters, 0.01))

Вероятность успешных прототипов на 500 итерациях с вероятностью 0.01 равна 0.008
Вероятность успешных прототипов на 1000 итерациях с вероятностью 0.01 равна 0.009
Вероятность успешных прототипов на 5000 итерациях с вероятностью 0.01 равна 0.0098


Получена ожидаемая картина. Но нам это не говорит прямо о том, что будет происходить именно на 201 прототипе у нашей компании. Давайте смоделируем и это.

In [58]:
def get_prob_201_prototype(win_rate: float) -> str:
    # первые 200 прототипов оказались неуспешны
    based_results = [0] * 200
    
    # список для итогового результата всех итераций
    results = []

    np.random.seed(42)
    for _ in range(500000):
        # добавляем результат 201 прототипа
        based_results.append(np.random.binomial(1, win_rate))
        # добавляем сумму всех значений в итоговый список
        results.append(sum(based_results))
        # возвращаем базовый список в исходное состояние
        based_results.pop()
    return f'Вероятность того, что 201-й прототип будет успешен: {sum(results) / len(results)}'

In [59]:
print(get_prob_201_prototype(0.005))

Вероятность того, что 201-й прототип будет успешен: 0.00496


In [60]:
print(get_prob_201_prototype(0.01))

Вероятность того, что 201-й прототип будет успешен: 0.009856


При моделировании получили тот же результат. Будет ли вероятность действительно такой? Возможно, что нет. Это лишь модель, построенная на допущениях. В качестве альтернативного варианта можно воспользоваться формулой Бернулли, чтобы посчитать вероятность того, что среди 201 эксперимента вообще будет один успешный, но это не совсем подходит под условия задачи.

# Вывод
Итого мы попробовали оценить, с какой вероятностью 201-й прототип окажется успешным. Узнать это точно нельзя, слишком мало информации доступно, а связь с компанией-конкурентом будет скорее мифической без проверки. Более правильным подходом было бы оценить исторические данные нашей компании, чтобы понять, от чего стоит отталкиваться при анализе. Это будет более приближенной к истинности оценке. Либо брать статистику успешностей прототипов всей индустрии.